# Managing Azure Machine Learning Workspace

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create Azure Machine Learning workspace from Python SDK
- Get the details of a workspace from Python SDK
- Load a workspace from Python SDK using parameters
- Write details of a workspace into a config file
- Load a workspace from Python SDK using a saved config file

**Motivations** - The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. Ability to create and manage workspace is a prerequisite for any activity in Azure Machine Learning.

# 1. Create a workspace

## 1.1. Import the required libraries

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.identity import DefaultAzureCredential

## 1.2 Configure where workspace needs to be created.
Before creating a workspace, we need identifier parameters - a subscription and resource group. We will use these parameters to define where the Azure Machine Learning workspace.

In [ ]:
# Enter details of your subscription
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"

The `MLClient` from `azure.ml` will be used to create the workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# get a handle to the subscription

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group)

## 1.3 Create a basic workspace
To create a basic workspace, we will define the following attributes
- `name` - Name of the workspace
- `location` - The Azure [location](https://azure.microsoft.com/en-us/global-infrastructure/services/?products=machine-learning-service) for workspace. For e.g. eastus, westus etc.
- `display_name` - Display name of the workspace
- `description` - Description of the workspace
- `hbi_workspace` - Flag to define whether the workspace is a High Impact workspace
- `tags` - (Optional) Tags to help search/filter on workspace easily

Using the `MLClient` created earlier, we will create the workspace. This command will start workspace creation and provide a confirmation.

In [ ]:
# Creating a unique workspace name with current datetime to avoid conflicts
from azure.ai.ml.entities import Workspace
import datetime

basic_workspace_name = "mlw-basic-prod-" + datetime.datetime.now().strftime(
    "%Y%m%d%H%M"
)

ws_basic = Workspace(
    name=basic_workspace_name,
    location="eastus",
    display_name="Basic workspace-example",
    description="This example shows how to create a basic workspace",
    hbi_workspace=False,
    tags=dict(purpose="demonstration"),
)

ws_basic = ml_client.workspaces.begin_create(ws_basic).result()
print(ws_basic)

## 1.4 Create a workspace with existing resources
When an Azure Machine Learning workspace is created, a default `storage account`, `container registry`, `key vault` and `application insights` are created. However, users can decide not to use the defaults and instead use their own assets for these. In this example we will create a workspace where we will pass in details of existing assets for the following:
- storage account
- container registry
- key vault
- application insights

Using the `MLClient` created earlier, we will create the workspace. This command will start workspace creation and provide a confirmation.

In [ ]:
# Creating a unique workspace name with current datetime to avoid conflicts
import datetime
from azure.ai.ml.entities import Workspace

basic_ex_workspace_name = "mlw-basicex-prod-" + datetime.datetime.now().strftime(
    "%Y%m%d%H%M"
)

# Change the following variables to resource ids of your existing storage account, key vault, application insights
# and container registry. Here we reuse the ones we just created for the basic workspace
existing_storage_account = (
    # e.g. "/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.Storage/storageAccounts/<STORAGE_ACCOUNT>"
    ws_basic.storage_account
)
existing_container_registry = (
    # e.g. "/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.ContainerRegistry/registries/<CONTAINER_REGISTRY>"
    ws_basic.container_registry
)
existing_key_vault = (
    # e.g. "/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.KeyVault/vaults/<KEY_VAULT>"
    ws_basic.key_vault
)
existing_application_insights = (
    # e.g. "/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.insights/components/<APP_INSIGHTS>"
    ws_basic.application_insights
)

ws_with_existing_resources = Workspace(
    name=basic_ex_workspace_name,
    location="eastus",
    display_name="Bring your own dependent resources-example",
    description="This sample specifies a workspace configuration with existing dependent resources",
    storage_account=existing_storage_account,
    container_registry=existing_container_registry,
    key_vault=existing_key_vault,
    application_insights=existing_application_insights,
    tags=dict(purpose="demonstration"),
)

ws_with_existing_resources = ml_client.begin_create_or_update(
    ws_with_existing_resources
).result()

print(ws_with_existing_resources)

## 1.5 Create a workspace for use with Azure Private Link
When using private link, your workspace cannot use the regular Azure Container Registry tasks compute for image building. Hence, you must set the image_build_compute property to some other CPU compute cluster name to use for Docker image environment building. You can also specify whether the private link workspace should be accessible over the internet using the public_network_access property. After workspace creation, create a private link endpoint for your workspace. For more details, see [Secure Azure Machine Learning workspace resources using virtual networks (VNets)](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-network-security-overview).

In [ ]:
# Creating a unique workspace name with current datetime to avoid conflicts
import datetime
from azure.ai.ml.entities import Workspace

basic_private_link_workspace_name = (
    "mlw-privatelink-prod-" + datetime.datetime.now().strftime("%Y%m%d%H%M")
)

ws_private = Workspace(
    name=basic_private_link_workspace_name,
    location="eastus",
    display_name="Private Link endpoint workspace-example",
    description="When using private link, you must set the image_build_compute property to a cluster name to use for Docker image environment building. You can also specify whether the workspace should be accessible over the internet.",
    image_build_compute="cpu-compute",
    public_network_access="Disabled",
    tags=dict(purpose="demonstration"),
)

ml_client.workspaces.begin_create(ws_private).result()

## 1.6 Create a workspace with multiple user assigned identities
For provisioning workspace with user-assigned managed identity and using it for auth between workspace and its associated resources, user needs to specify one or more user-assigned managed identities for the workspace, and select one of them as primary one. If no identity is specified, the system-assigned managed identity is the default.

In [ ]:
# Creating a unique workspace name with current datetime to avoid conflicts
import datetime
from azure.ai.ml.entities import Workspace
from azure.ai.ml.entities._credentials import (
    IdentityConfiguration,
    ManagedIdentityConfiguration,
)

basic_uai_workspace_name = "mlw-uai-prod-" + datetime.datetime.now().strftime(
    "%Y%m%d%H%M"
)
res_id1 = "<UAI RESOURCE ID 1>"
res_id2 = "<UAI RESOURCE ID 2>"
UAIs = [
    ManagedIdentityConfiguration(resource_id=res_id1),
    ManagedIdentityConfiguration(resource_id=res_id2),
]
managed_idenity = IdentityConfiguration(
    type="user_assigned", user_assigned_identities=UAIs
)

ws_uai = Workspace(
    name=basic_uai_workspace_name,
    location="eastus",
    display_name="UAI workspace-example",
    description="UAI workspace with multiple user assigned identities, and a primary user assigned identity.",
    tags=dict(purpose="demonstration"),
    identity=managed_idenity,
    primary_user_assigned_identity=res_id2,
)

ml_client.workspaces.begin_create(ws_uai).result()

# 2. Finding and loading workspace
Once created, we will examine how to get a list of workspaces and how to load them

## 2.1 Configure MLClient
To start with we will initialize the `MLClient` with a subscription ID and Resource Group Name. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.identity import DefaultAzureCredential

# Enter details of your subscription
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"

my_ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group)

## 2.2 Get a list of workspaces in a resource group
The `MLClient` can now be used to retrieve a list of workspaces within a group as shown below. 

In [ ]:
for ws in my_ml_client.workspaces.list():
    print(ws.name, ":", ws.location, ":", ws.description)

## 2.3 Get details of a specific workspace with name
The `MLClient` can also be used to get the details of a specific workspace.

In [ ]:
ws = my_ml_client.workspaces.get("<AML_WORKSPACE_NAME>")
# uncomment this line after providing a workspace name above
# print(ws.location,":", ws.resource_group)

# 3. Load Workspace
The `MLClient` can be used to load a workspace for use. 

## 3.1 Load a specific workspace using parameters
In this example we will load a specific workspace using parameters. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.identity import DefaultAzureCredential

ws = MLClient(
    DefaultAzureCredential(),
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group_name="<RESOURCE_GROUP>",
    workspace_name="<AML_WORKSPACE_NAME>",
)
print(ws.workspace_name)

## 3.2 Load workspace from a config file
The config details of a workspace can be saved to a file from the Azure Machine Learning [portal](https://ml.azure.com/). Click on the name of the portal on the top right corner to see the link to save the config file.

This config file can be used to load a workspace using `MLClient`. If no path is mentioned, path is defaulted to current folder. If no file name is mentioned, file name will be defaulted to `config.json`

In [ ]:
import json
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# create a demo config file "my_config.json" to be used in this example
# Data to be written
wsconfig = {
    "subscription_id": subscription_id,
    "resource_group": resource_group,
    "workspace_name": ws_basic.name,
}
with open("my_config.json", "w") as outfile:
    json.dump(wsconfig, outfile)

# read the config from the current directory
ws_from_config = MLClient.from_config(
    DefaultAzureCredential(), file_name="my_config.json"
)
print(ws_from_config.workspace_name)

# Clean up resources
Clean up the workspaces we created with async way(no wait mode).<br>
(If you want to delete the workspace with sync way, then run: ml_client.workspaces.begin_delete(ws_obj).result())

In [ ]:
ml_client.workspaces.begin_delete(name=ws_basic.name, delete_dependent_resources=True)
ml_client.workspaces.begin_delete(
    name=ws_with_existing_resources.name, delete_dependent_resources=False
)
ml_client.workspaces.begin_delete(name=ws_private.name, delete_dependent_resources=True)
ml_client.workspaces.begin_delete(name=ws_uai.name, delete_dependent_resources=True)